In [11]:
import sqlite3
import matplotlib.pyplot as plt

def read_db(filename, tst):
    connection = sqlite3.connect(filename)
    cursor = connection.cursor()

    cursor.execute("SELECT * FROM density_table WHERE timestamp = " + str(tst))
    
    return list(cursor)

def tst_range(filename):
    connection = sqlite3.connect(filename)
    cursor = connection.cursor()

    cursor.execute("SELECT DISTINCT timestamp FROM density_table WHERE 1 = 1 ORDER BY timestamp ASC")
    
    return list(cursor)
    

def show_for_tst(filename, tst, ax):
    dev = read_db(filename, tst)
    
    if len(dev) == 0:
        return

    northing = [d[1] for d in dev]
    easting  = [d[2] for d in dev]
    
    minN = min(northing) - 25
    minE = min(easting) - 25
    maxN = minN + 100
    maxE = minE + 100

    colors = [d[4] for d in dev]
    
    ax.set_xlim(minE, maxE)
    ax.set_ylim(minN, maxN)
    
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    
    max_color = 3 if max(colors) >= 1 else 0.6

    sc = ax.scatter(easting, northing, zorder=1, marker='s', s=130, c=colors, cmap='cool', vmin=0, vmax=max_color)
    plt.colorbar(sc, ax=ax)

    # Create sum (distinct northing easting)
    densSum = 0
    
    for n in range(minN, maxN):
        for e in range(minE, maxE):
            
            for dat in dev:
                if dat[1] == n and dat[2] == e:
                    densSum += dat[4]
                    break
    
def run(offset):
    _, subplots = plt.subplots(1, 3, figsize = (15, 4.5))
    
    tst = [t[0] for t in tst_range("scen2_dev1")][offset]
    
    show_for_tst("scen2_dev1", tst, subplots[0])
    show_for_tst("scen2_dev2", tst, subplots[1])
    show_for_tst("scen2_dev4", tst, subplots[2])
            

In [12]:
from ipywidgets import interact, interactive, fixed, interact_manual, widgets

interact(run, offset=widgets.IntSlider(min=0, max=200, step=1, value=0));

interactive(children=(IntSlider(value=0, description='offset', max=200), Output()), _dom_classes=('widget-inte…

In [3]:
def density_data(data, minN, minE, maxN, maxE):
    densities = []
    
    for n in range(minN, maxN):
        for e in range(minE, maxE):
            for dat in data:
                if dat[1] == n and dat[2] == e:
                    densities.append((n, e, dat[4]))
                    break
                    
    return densities

def count_differences(filename1, filename2):
    tsts = tst_range(filename1)
    
    res_eq = 0
    res_diff1 = 0
    res_diff2 = 0
    res_diff3 = 0
    res_sum = 0
    
    for tst in tsts:
        d1 = read_db(filename1, tst[0])
        d2 = read_db(filename2, tst[0])
        
        if len(d1) == 0 or len(d2) == 0:
            continue
            
        n1 = [d[1] for d in d1]
        e1  = [d[2] for d in d1]
            
        d1dens = density_data(d1, min(n1), min(e1), max(n1), max(e1))
        d2dens = density_data(d2, min(n1), min(e1), max(n1), max(e1))
        
        intersection = list(set(d1dens) & set(d2dens)) 
        err = len(d1dens) - len(intersection)
        
        if err == 0:
            res_eq += 1
        elif err == 1:
            res_diff1 += 1
        elif err == 2:
            res_diff2 += 1
        elif err == 3:
            res_diff3 += 1
            
        res_sum += 1
    
    return (res_sum, res_eq, res_diff1, res_diff2, res_diff3)

count_differences("scen2_dev1", "scen2_dev2")
        
    

OperationalError: no such table: density_table